特徴量を変えて精度比較
- パターン2で精度が上がるものは少なさそうなので特徴量に入れなくて良いかも

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import datetime
from datetime import datetime, timedelta
import json
import itertools
from tqdm import tqdm
import os
os.chdir("/content/drive/MyDrive/projects/Store_Sales-Time_Series_Forecasting/")
from models.model import LinearRegression_log, Ridge_log

In [2]:
conf = json.load(open("config/config.json"))
conf_model = json.load(open("config/config_ridge_log.json"))
conf.update(conf_model)
#cutoffs = conf["periods"]
all_params = [
    dict(zip(conf["cv_params"].keys(), v)) for v in itertools.product(*conf["cv_params"].values())
]
conf

{'exclude_features': ['avg_oil'],
 'cv_periods': {'other': [{'end': '2017-05-31', 'start': '2017-04-21'},
   {'end': '2017-06-30', 'start': '2017-06-01'},
   {'end': '2017-07-31', 'start': '2017-07-01'}]},
 'test_periods': {'other': {'train_start': '2017-04-21',
   'train_end': '2017-07-31',
   'test_start': '2017-08-01',
   'test_end': '2017-08-15'}},
 'sub_periods': {'other': {'train_start': '2017-04-21',
   'train_end': '2017-08-15',
   'sub_start': '2017-08-16',
   'sub_end': '2017-08-31'}},
 'model': 'Ridge_log',
 'cv_params': {'alpha': [0.01, 0.1, 1.0, 10]},
 'params': {'n_jobs': -1}}

In [3]:
# 特徴量
# upginiパターン1
df_add_f1 = pd.read_csv("store-sales-time-series-forecasting/train_upgini.csv", index_col=0, parse_dates=True)
# upginiパターン2
df_add_f2 = pd.read_csv("store-sales-time-series-forecasting/train_add_f_AllPeriods.csv", index_col=0, parse_dates=True)


# 使う特徴量を選択
ex_features = conf["exclude_features"]
df_add_f1 = df_add_f1.drop(ex_features, axis=1)
df_add_f2 = df_add_f2.drop(ex_features, axis=1)

In [4]:
df_add_f1.columns

Index(['dcoilwtico', 'dayofweek_cos', 'dayofweek_sin', 'wd', 'type_Additional',
       'type_Bridge', 'type_Event', 'type_Holiday', 'type_Transfer',
       'type_Work Day', 'wageday', 'school_season', 'f_year_cos1_cd165f8c',
       'f_year_cos3_8db0059c', 'f_finance_umap_4_d6d95b66',
       'f_mlending_approve_score_d4c33397'],
      dtype='object')

In [5]:
df_add_f2.columns

Index(['dcoilwtico', 'dayofweek_cos', 'dayofweek_sin', 'wd', 'type_Additional',
       'type_Bridge', 'type_Event', 'type_Holiday', 'type_Transfer',
       'type_Work Day', 'wageday', 'school_season', 'f_cci_pca_1_0ca24b1e',
       'f_nasdaq_d309709a', 'f_week_sin1_a71d22f6',
       'f_days_from_election_e1441706'],
      dtype='object')

In [6]:
# trainデータ
df_train = pd.read_csv("store-sales-time-series-forecasting/train.csv", index_col=1, parse_dates=True)
df_train.head()

,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0
2013-01-01,1,1,BABY CARE,0.0,0
2013-01-01,2,1,BEAUTY,0.0,0
2013-01-01,3,1,BEVERAGES,0.0,0
2013-01-01,4,1,BOOKS,0.0,0


In [7]:
df_test = pd.read_csv("store-sales-time-series-forecasting/test.csv", index_col=1, parse_dates=True)
df_test.head()

,id,store_nbr,family,onpromotion
date,,,,
2017-08-16,3000888,1,AUTOMOTIVE,0
2017-08-16,3000889,1,BABY CARE,0
2017-08-16,3000890,1,BEAUTY,2
2017-08-16,3000891,1,BEVERAGES,20
2017-08-16,3000892,1,BOOKS,0


In [8]:
def walk_forward_validation(df_X, df_y , model, cutoffs):
  indexes = []
  y_vals = []
  y_preds =[]  
  for i in range(len(cutoffs)-1):
    train_start = cutoffs[i]["start"]
    train_end = cutoffs[i]["end"]
    val_start = cutoffs[i+1]["start"]
    val_end = cutoffs[i+1]["end"]

    X_train = df_X.loc[train_start : train_end]
    y_train = df_y.loc[train_start : train_end]
    X_val = df_X.loc[val_start : val_end]
    y_val = df_y.loc[val_start : val_end] 
    y_vals.extend(y_val)

    # モデルの学習
    model.fit(X_train, y_train)
    # 検証データの予測
    y_preds.extend(model.predict(X_val))
    indexes.extend(df_y.loc[val_start : val_end].index)

  df_cv = pd.DataFrame({"y" : y_vals, "y_pred" : y_preds}, index=indexes)
  return df_cv

学習期間は固定して、特徴量の選択パターンだけ変えてみる
- upginiパターン1
- upginiパターン2
- upiginiで作ったものは使わない

In [25]:
from sklearn.metrics import mean_squared_log_error

result = []
sub = []
cutoffs = conf["cv_periods"]

for family in tqdm(df_train["family"].unique()):
  for nbr in df_train["store_nbr"].unique():
    df_y = df_train.query(" (family==@family) & (store_nbr==@nbr) ").sales
    rmsles = []
    for params in all_params:
      if family in cutoffs:
        cutoff = cutoffs[family]
      else:
        cutoff = cutoffs["other"]
      
      model = Ridge_log(**params)
      df_cv = walk_forward_validation(df_add_f1, df_y , model, cutoff)

      y_true = df_cv["y"]
      y_pred = df_cv["y_pred"]
      y_pred[y_pred<0] = 0 # 予測が0未満の場合は0に置換

      rmsles.append(mean_squared_log_error(y_true, y_pred, squared=False))
    
    best_params = pd.DataFrame({"all_params" : all_params, "rmsles" : rmsles}).sort_values("rmsles").iloc[0].to_list()
    best_params.extend([family, nbr])
    result.append(best_params)

df_result1 = pd.DataFrame(result)
df_result1.columns = ["params", "val_RMSLE", "family", "store_nbr"]
df_result1 = df_result1.sort_values(["store_nbr", "family"])
df_result1 = df_result1.reset_index(drop=True)
df_result1.head()

100%|██████████| 33/33 [06:32<00:00, 11.89s/it]


,params,val_RMSLE,family,store_nbr
0,{'alpha': 10},0.622882,AUTOMOTIVE,1
1,{'alpha': 0.01},0.000000,BABY CARE,1
2,{'alpha': 10},0.482125,BEAUTY,1
3,{'alpha': 0.1},0.248547,BEVERAGES,1
4,{'alpha': 0.01},0.448476,BOOKS,1


In [26]:
from sklearn.metrics import mean_squared_log_error

result = []
sub = []
cutoffs = conf["cv_periods"]

for family in tqdm(df_train["family"].unique()):
  for nbr in df_train["store_nbr"].unique():
    df_y = df_train.query(" (family==@family) & (store_nbr==@nbr) ").sales
    rmsles = []
    for params in all_params:
      if family in cutoffs:
        cutoff = cutoffs[family]
      else:
        cutoff = cutoffs["other"]
      
      model = Ridge_log(**params)
      df_cv = walk_forward_validation(df_add_f2, df_y , model, cutoff)

      y_true = df_cv["y"]
      y_pred = df_cv["y_pred"]
      y_pred[y_pred<0] = 0 # 予測が0未満の場合は0に置換

      rmsles.append(mean_squared_log_error(y_true, y_pred, squared=False))
    
    best_params = pd.DataFrame({"all_params" : all_params, "rmsles" : rmsles}).sort_values("rmsles").iloc[0].to_list()
    best_params.extend([family, nbr])
    result.append(best_params)

df_result2 = pd.DataFrame(result)
df_result2.columns = ["params", "val_RMSLE", "family", "store_nbr"]
df_result2 = df_result2.sort_values(["store_nbr", "family"])
df_result2 = df_result2.reset_index(drop=True)
df_result2.head()

100%|██████████| 33/33 [06:28<00:00, 11.78s/it]


,params,val_RMSLE,family,store_nbr
0,{'alpha': 0.01},0.639982,AUTOMOTIVE,1
1,{'alpha': 0.01},0.000000,BABY CARE,1
2,{'alpha': 10},0.529729,BEAUTY,1
3,{'alpha': 10},0.262772,BEVERAGES,1
4,{'alpha': 1.0},0.448476,BOOKS,1


In [27]:
drop_ex = [
    'f_year_cos1_cd165f8c',
    'f_year_cos3_8db0059c', 
    'f_finance_umap_4_d6d95b66',
    'f_mlending_approve_score_d4c33397'
    ]

df_add_f3 = df_add_f1.drop(drop_ex, axis=1)
df_add_f3.head()

,dcoilwtico,dayofweek_cos,dayofweek_sin,wd,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,wageday,school_season
date,,,,,,,,,,,,
2017-04-21,49.640000,-0.222521,-9.749279e-01,1,0,0,0,0,0,0,0,1
2017-04-22,49.393333,0.623490,-7.818315e-01,0,0,0,0,0,0,0,0,1
2017-04-23,49.146667,1.000000,-2.449294e-16,0,0,0,0,0,0,0,0,1
2017-04-24,48.900000,0.623490,7.818315e-01,1,0,0,0,0,0,0,0,1
2017-04-25,49.220000,-0.222521,9.749279e-01,1,0,0,0,0,0,0,0,1


In [28]:
from sklearn.metrics import mean_squared_log_error

result = []
sub = []
cutoffs = conf["cv_periods"]

for family in tqdm(df_train["family"].unique()):
  for nbr in df_train["store_nbr"].unique():
    df_y = df_train.query(" (family==@family) & (store_nbr==@nbr) ").sales
    rmsles = []
    for params in all_params:
      if family in cutoffs:
        cutoff = cutoffs[family]
      else:
        cutoff = cutoffs["other"]
      
      model = Ridge_log(**params)
      df_cv = walk_forward_validation(df_add_f3, df_y , model, cutoff)

      y_true = df_cv["y"]
      y_pred = df_cv["y_pred"]
      y_pred[y_pred<0] = 0 # 予測が0未満の場合は0に置換

      rmsles.append(mean_squared_log_error(y_true, y_pred, squared=False))
    
    best_params = pd.DataFrame({"all_params" : all_params, "rmsles" : rmsles}).sort_values("rmsles").iloc[0].to_list()
    best_params.extend([family, nbr])
    result.append(best_params)

df_result3 = pd.DataFrame(result)
df_result3.columns = ["params", "val_RMSLE", "family", "store_nbr"]
df_result3 = df_result3.sort_values(["store_nbr", "family"])
df_result3 = df_result3.reset_index(drop=True)
df_result3.head()

100%|██████████| 33/33 [06:27<00:00, 11.73s/it]


,params,val_RMSLE,family,store_nbr
0,{'alpha': 10},0.622977,AUTOMOTIVE,1
1,{'alpha': 0.01},0.000000,BABY CARE,1
2,{'alpha': 10},0.502513,BEAUTY,1
3,{'alpha': 1.0},0.250717,BEVERAGES,1
4,{'alpha': 10},0.529050,BOOKS,1


In [29]:
a1 = df_result1.groupby(["family"]).mean().sort_values("val_RMSLE", ascending=False)
a1.columns = ["val_RMSLE1",	"store_nbr"]
a2 = df_result2.groupby(["family"]).mean().sort_values("val_RMSLE", ascending=False)
a2.columns = ["val_RMSLE2",	"store_nbr"]
a3 = df_result3.groupby(["family"]).mean().sort_values("val_RMSLE", ascending=False)
a3.columns = ["val_RMSLE3",	"store_nbr"]

In [30]:
val_result = pd.concat([a1.drop("store_nbr", axis=1), a2.drop("store_nbr", axis=1), a3.drop("store_nbr", axis=1)], axis=1)
val_result.idxmin(axis=1).value_counts()

val_RMSLE3    20
val_RMSLE1    13
dtype: int64

In [31]:
val_result.idxmin(axis=1)

family
LINGERIE                      val_RMSLE1
GROCERY II                    val_RMSLE1
HARDWARE                      val_RMSLE3
BEAUTY                        val_RMSLE1
CELEBRATION                   val_RMSLE3
AUTOMOTIVE                    val_RMSLE3
SEAFOOD                       val_RMSLE3
MAGAZINES                     val_RMSLE3
HOME AND KITCHEN I            val_RMSLE1
LIQUOR,WINE,BEER              val_RMSLE3
PLAYERS AND ELECTRONICS       val_RMSLE1
PET SUPPLIES                  val_RMSLE3
HOME AND KITCHEN II           val_RMSLE1
SCHOOL AND OFFICE SUPPLIES    val_RMSLE1
LADIESWEAR                    val_RMSLE3
HOME APPLIANCES               val_RMSLE1
LAWN AND GARDEN               val_RMSLE3
EGGS                          val_RMSLE3
FROZEN FOODS                  val_RMSLE1
PREPARED FOODS                val_RMSLE1
PERSONAL CARE                 val_RMSLE3
CLEANING                      val_RMSLE1
MEATS                         val_RMSLE3
POULTRY                       val_RMSLE3
BABY CARE

In [33]:
a = val_result.idxmin(axis=1)
d = dict()
for i, v in zip(a.index, a.values):
  d.update({i : v})
d

{'LINGERIE': 'val_RMSLE1',
 'GROCERY II': 'val_RMSLE1',
 'HARDWARE': 'val_RMSLE3',
 'BEAUTY': 'val_RMSLE1',
 'CELEBRATION': 'val_RMSLE3',
 'AUTOMOTIVE': 'val_RMSLE3',
 'SEAFOOD': 'val_RMSLE3',
 'MAGAZINES': 'val_RMSLE3',
 'HOME AND KITCHEN I': 'val_RMSLE1',
 'LIQUOR,WINE,BEER': 'val_RMSLE3',
 'PLAYERS AND ELECTRONICS': 'val_RMSLE1',
 'PET SUPPLIES': 'val_RMSLE3',
 'HOME AND KITCHEN II': 'val_RMSLE1',
 'SCHOOL AND OFFICE SUPPLIES': 'val_RMSLE1',
 'LADIESWEAR': 'val_RMSLE3',
 'HOME APPLIANCES': 'val_RMSLE1',
 'LAWN AND GARDEN': 'val_RMSLE3',
 'EGGS': 'val_RMSLE3',
 'FROZEN FOODS': 'val_RMSLE1',
 'PREPARED FOODS': 'val_RMSLE1',
 'PERSONAL CARE': 'val_RMSLE3',
 'CLEANING': 'val_RMSLE1',
 'MEATS': 'val_RMSLE3',
 'POULTRY': 'val_RMSLE3',
 'BABY CARE': 'val_RMSLE3',
 'PRODUCE': 'val_RMSLE3',
 'HOME CARE': 'val_RMSLE1',
 'BEVERAGES': 'val_RMSLE3',
 'DELI': 'val_RMSLE3',
 'GROCERY I': 'val_RMSLE3',
 'DAIRY': 'val_RMSLE3',
 'BREAD/BAKERY': 'val_RMSLE3',
 'BOOKS': 'val_RMSLE1'}

In [18]:
df_result1.columns = ["params",	"val_RMSLE1",	"family",	"store_nbr"]
df_result2.columns = ["params",	"val_RMSLE2",	"family",	"store_nbr"]
df_result3.columns = ["params",	"val_RMSLE3",	"family",	"store_nbr"]
val_result = pd.concat([df_result1["val_RMSLE1"], df_result2["val_RMSLE2"], df_result3["val_RMSLE3"]], axis=1)
val_result.idxmin(axis=1).value_counts()

val_RMSLE3    824
val_RMSLE1    779
val_RMSLE2    179
dtype: int64